In [2]:
import numpy as np
import pandas as pd
import json
from pandas.io.json import json_normalize # transorm json into a dataframe
from geopy.geocoders import Nominatim #converting address into longitude and latitude
import requests # for requests
from bs4 import BeautifulSoup # for scraping
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans


### scraping data from wikipedia and storing in a dataframe

In [3]:
data=requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text # sending GET request

In [4]:
bs=BeautifulSoup(data,'html.parser') #parsing data from html

In [5]:
PostalCodeList=[]
BoroughList=[]
NeighborhoodList=[]

In [6]:
for row in bs.find('table').find_all('tr'):  #finding all the rows in the table and append them into the lists
    cells=row.find_all('td')
    if(len(cells)>0):
        PostalCodeList.append(cells[0].text)
        BoroughList.append(cells[1].text)
        NeighborhoodList.append(cells[2].text.rstrip('\n')) # stripping new lines in the neighborhood cell

In [9]:
Toronto_df=pd.DataFrame({"PostalCode": PostalCodeList, "Borough":BoroughList, "Neighborhood": NeighborhoodList})  #creating a new dataframe with the lists
Toronto_df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


### Dropping the "Not assigned" cells in borough

In [12]:
Toronto_df_dropna=Toronto_df[Toronto_df.Borough!="Not assigned"].reset_index(drop=True)
Toronto_df_dropna.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor


### Grouping neighborhoods in the same borough

In [13]:
Toronto_df_grouped=Toronto_df_dropna.groupby(["PostalCode", "Borough"], as_index=False).agg(lambda x: ", ".join(x))
Toronto_df_grouped.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### Make the value same as Borough for Neighborhood="Not assigned"

In [15]:
for index, row in Toronto_df_grouped.iterrows():
    if row["Neighborhood"]=="Not assigned":
        row["Neighborhood"]=row["Borough"]
Toronto_df_grouped.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### Checking whether the dataframe is same as shown in the question

In [18]:
column=["PostalCode", "Borough", "Neighborhood"]
test_df=pd.DataFrame(columns=column)
test_list=["M5G","M2H","M4B","M1J","M4G","M4M","M1R","M9V","M9L","M5V","M1B","M5A"]
for postcode in test_list:
    test_df=test_df.append(Toronto_df_grouped[Toronto_df_grouped["PostalCode"]==postcode], ignore_index=True)
test_df

,PostalCode,Borough,Neighborhood
0,M5G,Downtown Toronto,Central Bay Street
1,M2H,North York,Hillcrest Village
2,M4B,East York,"Woodbine Gardens, Parkview Hill"
3,M1J,Scarborough,Scarborough Village
4,M4G,East York,Leaside
5,M4M,East Toronto,Studio District
6,M1R,Scarborough,"Maryvale, Wexford"
7,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ..."
8,M9L,North York,Humber Summit
9,M5V,Downtown Toronto,"CN Tower, Bathurst Quay, Island airport, Harbo..."


### Print the number of rows of the cleaned dataframe

In [19]:
Toronto_df_grouped.shape

(103, 3)